In [1]:
import numpy as np
import argparse
import os
import sys
import torch
import torch.fft as F
from importlib import reload
from torch.nn.functional import relu
import torch.nn as nn
import torch.nn.functional as Func
import torch.optim as optim
import utils
from matplotlib import pyplot as plt
import random
import copy

from utils import kplot,mask_naiveRand,mask_filter
from mnet import MNet

from mask_backward_new import mask_backward, mask_eval
from utils import mask_complete , mask_makebinary,kplot, mask_filter, mask_makebinary,raw_normalize

sys.path.insert(0,'/home/huangz78/mri/unet/')
from unet_model import UNet

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [2]:
reload(utils)
from utils import mask_complete , mask_makebinary,kplot, mask_filter, mask_makebinary,raw_normalize


In [3]:
# load a mnet
mnet = MNet(out_size=320-24)
checkpoint = torch.load('/home/huangz78/checkpoints/mnet.pth')
mnet.load_state_dict(checkpoint['model_state_dict'])
print('MNet loaded successfully from: ' + '/home/huangz78/checkpoints/mnet.pth')

MNet loaded successfully from: /home/huangz78/checkpoints/mnet.pth


In [4]:
# load a unet for maskbackward
UNET =  UNet(n_channels=1,n_classes=1,bilinear=True,skip=False)
checkpoint = torch.load('/home/huangz78/checkpoints/unet_'+ str(UNET.n_channels) +'.pth')
UNET.load_state_dict(checkpoint['model_state_dict'])
print('Unet loaded successfully from : ' + '/home/huangz78/checkpoints/unet_'+ str(UNET.n_channels) +'.pth' )

Unet loaded successfully from : /home/huangz78/checkpoints/unet_1.pth


In [5]:
train_dir = '/home/huangz78/data/traindata_x.npz'
# train_sub = np.load(train_dir)['x']
train_full = np.load(train_dir)['xfull']

In [6]:
test_dir = '/home/huangz78/data/testdata_x.npz'
# test_sub  = torch.tensor(np.load(test_dir)['x'])     ; test_sub  = test_sub[0:10,:,:]
test_full = torch.tensor(np.load(test_dir)['xfull']) 

In [7]:
fullmask  = torch.tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],dtype=torch.float)

In [8]:
def alternating_update_with_unetRecon(mnet,unet,trainfulls,testimg,mask_init,mask_init_full=True,\
                                      maxIter_mb=50,evalmode='unet',maxRep=5,alpha=2.8*1e-5,lr_mb=1e-3,\
                                      lr_mn=1e-3,\
                                      corefreq=24,budget=56,dtyp=torch.float,verbose=False):
    if mask_init_full:
        fullmask = torch.tensor(mask_init).clone()
        highmask = mask_filter(fullmask,base=corefreq,roll=True)
    else:
        fullmask = mask_complete(torch.tensor(mask_init),xstar.shape[0],rolled=True,dtyp=dtyp)
        highmask = torch.tensor(mask_init).clone()
    DTyp = torch.cfloat if dtyp==torch.float else torch.cdouble
    criterion_mnet = nn.BCEWithLogitsLoss()
    optimizer_m = optim.RMSprop(mnet.parameters(), lr=lr_mn, weight_decay=0, momentum=0)
    unet_eval = UNet(n_channels=1,n_classes=1,bilinear=True,skip=False)
    unet_eval = copy.deepcopy(unet)
    unet_eval.eval()
    # training loop
    for xstar in trainfulls:
        xstar = torch.tensor(xstar,dtype=dtyp)
        yfull = torch.fft.fftshift(F.fftn(xstar,dim=(0,1),norm='ortho')) # y is ROLLED!
        
      ## (1) mask_backward
        highmask_refined,unet = mask_backward(highmask,xstar,unet=unet,\
                          beta=1.,alpha=alpha,maxIter=maxIter_mb,seed=0,break_limit=maxIter_mb//2,\
                          lr=lr_mb,mode='UNET',\
                          verbose=verbose,dtyp=torch.float)
        
      ## (2) update mnet
        lowfreqmask,_,_ = mask_naiveRand(xstar.shape[0],fix=corefreq,other=0,roll=True)
        z_lf = torch.fft.ifftshift(torch.tensordot(torch.diag(lowfreqmask).to(DTyp),yfull,dims=([1],[0])))
        x_lf = torch.abs(F.ifftn(z_lf,dim=(0,1),norm='ortho'))
        mnet.train()
        rep = 0
        while rep < maxRep:
            mask_pred   = mnet(x_lf.view(1,1,xstar.shape[0],xstar.shape[1]))
            train_loss  = criterion_mnet(mask_pred,highmask_refined.view(mask_pred.shape))
            optimizer_m.zero_grad()
            train_loss.backward()
            optimizer_m.step()
            rep += 1
        mnet.eval()
        
      ## (3) check mnet performance: does it beat random sampling?
        testimg    = torch.tensor(testimg,dtype=dtyp)
        yfull_test = torch.fft.fftshift(F.fftn(testimg,dim=(0,1),norm='ortho'))
        z_test_lf  = torch.fft.ifftshift(torch.tensordot(torch.diag(lowfreqmask).to(DTyp),yfull_test,dims=([1],[0])))
        x_test_lf  = torch.abs(F.ifftn(z_test_lf,dim=(0,1),norm='ortho'))
        
        highmask_tmp  = torch.sigmoid( mnet(x_test_lf.view(1,1,testimg.shape[0],testimg.shape[1])).view(-1) )   
        highmask_test = mask_makebinary( raw_normalize(highmask_tmp,budget) , sigma=False )
        
        mask_rand,_,_  = mask_naiveRand(xstar.shape[0],fix=corefreq,other=highmask_test.sum(),roll=True)
        mask_test  = mask_complete(highmask_test.view(-1),xstar.shape[0],rolled=True,dtyp=dtyp)
        
        
        print('Quality of random mask : ', mask_eval(mask_rand,testimg,UNET=unet_eval))
        print('Quality of mnet   mask : ', mask_eval(mask_test,testimg,UNET=unet_eval))
        
    # return mnet, unet

In [9]:
alternating_update_with_unetRecon(mnet,UNET,train_full,test_full[0,:,:],fullmask,\
                                  maxIter_mb=50,verbose=True,alpha=2.9e-5)

/opt/anaconda/envs/pyenv/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/huangz78/mri/mask_backward_new.py:109: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  xstar = torch.tensor(xstar,dtype=dtyp); highmask = torch.tensor(highmask,dtype=dtyp)
/home/huangz78/mri/mask_backward_new.py:115: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  fullmask = torch.tensor( mask_complete(M_high,imgHeg,dtyp=dtyp) )
/opt/anaconda/envs/pyenv/lib/python3.7/site-packages/torch/autograd/__init__.py:147: U

iter: 1, upper level loss: 0.11352092027664185


/opt/anaconda/envs/pyenv/lib/python3.7/site-packages/torch/autograd/__init__.py:147: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /opt/conda/conda-bld/pytorch_1616554786529/work/aten/src/ATen/native/Copy.cpp:219.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


iter: 11, upper level loss: 0.07103174179792404
Iter 16, rows added: 0, rows reducted: 2
Iter 17, rows added: 0, rows reducted: 2
Iter 18, rows added: 20, rows reducted: 0
Iter 19, rows added: 8, rows reducted: 0
Iter 20, rows added: 10, rows reducted: 0
iter: 21, upper level loss: 0.03826776519417763
 changed rows in this batch: 42.0, loss of current mask: 8.767740428447723
samp. ratio: 0.328125, Recon. rel. err: 0.03470051661133766 

Iter 21, rows added: 4, rows reducted: 0
Iter 22, rows added: 5, rows reducted: 0
Iter 23, rows added: 2, rows reducted: 0
Iter 24, rows added: 2, rows reducted: 0
Iter 25, rows added: 1, rows reducted: 0
Iter 26, rows added: 0, rows reducted: 1
Iter 29, rows added: 0, rows reducted: 2
iter: 31, upper level loss: 0.025202447548508644
 changed rows in this batch: 17.0, loss of current mask: 8.000780642032623
samp. ratio: 0.3625, Recon. rel. err: 0.021122366189956665 

Iter 32, rows added: 0, rows reducted: 2
Iter 35, rows added: 1, rows reducted: 0
Iter 3

/opt/anaconda/envs/pyenv/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


> /home/huangz78/mri/utils.py(122)raw_normalize()
    120     '''
    121     breakpoint()
--> 122     if torch.sum(M>0.5) > budget:
    123         sampinds  = np.argsort(M.detach().numpy())[::-1][0:budget]
    124         eraseinds = np.setdiff1d(np.arange(0,M.shape[0],1),sampinds)

ipdb> n
> /home/huangz78/mri/utils.py(123)raw_normalize()
    121     breakpoint()
    122     if torch.sum(M>0.5) > budget:
--> 123         sampinds  = np.argsort(M.detach().numpy())[::-1][0:budget]
    124         eraseinds = np.setdiff1d(np.arange(0,M.shape[0],1),sampinds)
    125         M[eraseinds] = 0

ipdb> n
> /home/huangz78/mri/utils.py(124)raw_normalize()
    122     if torch.sum(M>0.5) > budget:
    123         sampinds  = np.argsort(M.detach().numpy())[::-1][0:budget]
--> 124         eraseinds = np.setdiff1d(np.arange(0,M.shape[0],1),sampinds)
    125         M[eraseinds] = 0
    126     return M

ipdb> n
> /home/huangz78/mri/utils.py(125)raw_normalize()
    123         sampinds  = np.argsort

ipdb> c
Quality of random mask :  0.13243875
Quality of refined mask:  0.261978
iter: 1, upper level loss: 0.1189175546169281

Program interrupted. (Use 'cont' to resume).
--Return--
tensor([[[[-1...NormBackward>)
> /opt/anaconda/envs/pyenv/lib/python3.7/site-packages/torch/nn/functional.py(2150)batch_norm()
   2148 
   2149     return torch.batch_norm(
-> 2150         input, weight, bias, running_mean, running_var, training, momentum, eps, torch.backends.cudnn.enabled
   2151     )
   2152 

ipdb> q


BdbQuit: 

In [ ]:
M = np.arange(0,100,2)
print(M>50)
np.sum(M>50)